
<div align="center">
  <a href="https://ultralytics.com/yolo" target="_blank">
    <img width="320" src="https://github.com/miteruel/ahhqdia/blob/main/Logozoo.png?raw=true">
  </a>

   | [Español](https://docs.ultralytics.com/es/) |

  <a href="https://github.com/ultralytics/ultralytics/actions/workflows/ci.yml"><img src="https://github.com/ultralytics/ultralytics/actions/workflows/ci.yml/badge.svg" alt="Ultralytics CI"></a>
  <a href="https://console.paperspace.com/github/ultralytics/ultralytics"><img src="https://assets.paperspace.io/img/gradient-badge.svg" alt="Run on Gradient"/></a>
  <a href="https://colab.research.google.com/github/ultralytics/ultralytics/blob/main/examples/tutorial.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
  <a href="https://www.kaggle.com/models/ultralytics/yolo11"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Kaggle"></a>

  <a href="https://ultralytics.com/discord"><img alt="Discord" src="https://img.shields.io/discord/1089800235347353640?logo=discord&logoColor=white&label=Discord&color=blue"></a>
  <a href="https://community.ultralytics.com"><img alt="Ultralytics Forums" src="https://img.shields.io/discourse/users?server=https%3A%2F%2Fcommunity.ultralytics.com&logo=discourse&label=Forums&color=blue"></a>
  <a href="https://reddit.com/r/ultralytics"><img alt="Ultralytics Reddit" src="https://img.shields.io/reddit/subreddit-subscribers/ultralytics?style=flat&logo=reddit&logoColor=white&label=Reddit&color=blue"></a>
</div>



# Setup

pip install `ultralytics` and [dependencies](https://github.com/ultralytics/ultralytics/blob/main/pyproject.toml) and check software and hardware.

[![PyPI - Version](https://img.shields.io/pypi/v/ultralytics?logo=pypi&logoColor=white)](https://pypi.org/project/ultralytics/) [![Downloads](https://static.pepy.tech/badge/ultralytics)](https://www.pepy.tech/projects/ultralytics) [![PyPI - Python Version](https://img.shields.io/pypi/pyversions/ultralytics?logo=python&logoColor=gold)](https://pypi.org/project/ultralytics/)

In [ ]:
#%pip install ultralytics
import ultralytics
ultralytics.checks()

gradio>=4.0.0
ultralytics>=8.0.0
opencv-python>=4.8.0
Pillow>=9.0.0
numpy>=1.21.0
torch>=1.11.0
torchvision>=0.12.0

In [ ]:
import gradio as gr
import cv2
from ultralytics import YOLO
import numpy as np
from PIL import Image
import time

class YOLODetector:
    def __init__(self, model_path='yolo11n.pt'):
        
        """
        
        model = YOLO('yolo11n.pt')  ## Modelo de detección de objetos preentrenado

        Inicializa el detector YOLO
        Args:
            model_path: Ruta al modelo YOLO (por defecto yolov8n.pt)
        """
        self.model = YOLO(model_path)
        self.detection_history = []
    
    def detect_objects(self, image):
        """
        Detecta objetos en una imagen
        Args:
            image: Imagen PIL o numpy array
        Returns:
            tuple: (imagen_con_detecciones, lista_objetos_detectados)
        """
        # Convertir PIL a numpy si es necesario
        if isinstance(image, Image.Image):
            image = np.array(image)
        
        # Realizar detección
        results = self.model(image)
        
        # Procesar resultados
        detected_objects = []
        annotated_image = image.copy()
        
        for result in results:
            boxes = result.boxes
            if boxes is not None:
                for box in boxes:
                    # Obtener coordenadas de la caja
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
                    
                    # Obtener confianza y clase
                    confidence = box.conf[0].cpu().numpy()
                    class_id = int(box.cls[0].cpu().numpy())
                    class_name = self.model.names[class_id]
                    
                    # Añadir a la lista de objetos detectados
                    detected_objects.append({
                        'objeto': class_name,
                        'confianza': f"{confidence:.2f}",
                        'coordenadas': f"({x1}, {y1}, {x2}, {y2})"
                    })
                    
                    # Dibujar caja y etiqueta en la imagen
                    cv2.rectangle(annotated_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    
                    label = f"{class_name}: {confidence:.2f}"
                    label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)[0]
                    cv2.rectangle(annotated_image, (x1, y1 - label_size[1] - 10), 
                                (x1 + label_size[0], y1), (0, 255, 0), -1)
                    cv2.putText(annotated_image, label, (x1, y1 - 5), 
                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
        
        # Actualizar historial
        self.detection_history.append({
            'timestamp': time.strftime("%H:%M:%S"),
            'count': len(detected_objects),
            'objects': detected_objects
        })
        
        # Mantener solo las últimas 10 detecciones
        if len(self.detection_history) > 10:
            self.detection_history = self.detection_history[-10:]
        
        return annotated_image, detected_objects

# Crear instancia del detector
detector = YOLODetector()

def process_image(image):
    """
    Procesa una imagen y devuelve resultados
    """
    if image is None:
        return None, "No se ha proporcionado ninguna imagen"
    
    try:
        # Detectar objetos
        annotated_image, detected_objects = detector.detect_objects(image)
        
        # Crear texto con objetos detectados
        if detected_objects:
            objects_text = f"🔍 **Objetos detectados ({len(detected_objects)}):**\n\n"
            for i, obj in enumerate(detected_objects, 1):
                objects_text += f"**{i}.** {obj['objeto']} (Confianza: {obj['confianza']})\n"
                objects_text += f"   📍 Coordenadas: {obj['coordenadas']}\n\n"
        else:
            objects_text = "❌ No se detectaron objetos en la imagen"
        
        return annotated_image, objects_text
        
    except Exception as e:
        return None, f"❌ Error al procesar la imagen: {str(e)}"

def get_detection_history():
    """
    Obtiene el historial de detecciones
    """
    if not detector.detection_history:
        return "📝 **Historial vacío**\n\nNo hay detecciones previas"
    
    history_text = "📝 **Historial de Detecciones:**\n\n"
    for i, detection in enumerate(reversed(detector.detection_history), 1):
        history_text += f"**{i}. {detection['timestamp']}** - {detection['count']} objetos\n"
        for obj in detection['objects']:
            history_text += f"   • {obj['objeto']} ({obj['confianza']})\n"
        history_text += "\n"
    
    return history_text

def clear_history():
    """
    Limpia el historial de detecciones
    """
    detector.detection_history = []
    return "✅ Historial limpiado", "📝 **Historial vacío**\n\nNo hay detecciones previas"

# Crear interfaz con Gradio
with gr.Blocks(title="🔍 Detector YOLO", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 🔍 Detector de Objetos con YOLO
    
    Sube una imagen o usa tu cámara para detectar objetos en tiempo real usando YOLOv8.
    """)
    
    with gr.Row():
        with gr.Column(scale=2):
            gr.Markdown("### 📷 Entrada de Imagen")
            
            # Crear pestañas para diferentes tipos de entrada
            with gr.Tab("📁 Subir Imagen"):
                input_image = gr.Image(
                    label="Seleccionar imagen",
                    type="pil"
                )
                upload_btn = gr.Button("🔍 Analizar Imagen", variant="primary")
            
            with gr.Tab("📹 Cámara Web"):
                camera_image = gr.Image(
                    label="Capturar desde cámara",
                    sources="webcam",
                    type="pil"
                )
                camera_btn = gr.Button("🔍 Analizar desde Cámara", variant="primary")
        
        with gr.Column(scale=2):
            gr.Markdown("### 🎯 Resultado")
            output_image = gr.Image(
                label="Imagen con detecciones",
                type="pil"
            )
    
    with gr.Row():
        with gr.Column():
            gr.Markdown("### 📋 Objetos Detectados")
            objects_output = gr.Markdown(
                value="Esperando imagen para analizar...",
                label="Lista de objetos"
            )
        
        with gr.Column():
            gr.Markdown("### 📝 Historial")
            history_output = gr.Markdown(
                value="📝 **Historial vacío**\n\nNo hay detecciones previas",
                label="Historial de detecciones"
            )
            
            with gr.Row():
                refresh_btn = gr.Button("🔄 Actualizar Historial", size="sm")
                clear_btn = gr.Button("🗑️ Limpiar Historial", size="sm", variant="secondary")
    
    # Información adicional
    with gr.Accordion("ℹ️ Información", open=False):
        gr.Markdown("""
        ### Sobre esta aplicación:
        - **Modelo**: YOLOv8 nano (rápido y eficiente)
        - **Clases detectables**: 80 clases del dataset COCO
        - **Formato de entrada**: JPG, PNG, WEBP
        - **Funciones**: Detección, visualización y historial
        
        ### Instrucciones:
        1. Sube una imagen o usa la cámara web
        2. Haz clic en "Analizar" para detectar objetos
        3. Ve los resultados con cajas delimitadoras
        4. Revisa la lista de objetos detectados
        5. Consulta el historial de detecciones anteriores
        """)
    
    # Conectar eventos
    upload_btn.click(
        fn=process_image,
        inputs=[input_image],
        outputs=[output_image, objects_output]
    ).then(
        fn=get_detection_history,
        outputs=[history_output]
    )
    
    camera_btn.click(
        fn=process_image,
        inputs=[camera_image],
        outputs=[output_image, objects_output]
    ).then(
        fn=get_detection_history,
        outputs=[history_output]
    )
    
    refresh_btn.click(
        fn=get_detection_history,
        outputs=[history_output]
    )
    
    clear_btn.click(
        fn=clear_history,
        outputs=[objects_output, history_output]
    )

# Configuración para ejecutar la aplicación
if __name__ == "__main__":
    # Instalar dependencias si es necesario
    try:
        import ultralytics
        print("✅ Ultralytics YOLO está instalado")
    except ImportError:
        print("❌ Instalando ultralytics...")
        import subprocess
        subprocess.run(["pip", "install", "ultralytics"])
    
    print("🚀 Iniciando aplicación YOLO Detector...")
    print("📱 La aplicación estará disponible en: http://localhost:7860")
    
    # Lanzar la aplicación
    demo.launch(
        server_name="0.0.0.0",  # Permite acceso desde otras direcciones IP
        server_port=7862,       # Puerto por defecto
        share=False,            # Cambiar a True para compartir públicamente
        debug=True              # Modo debug para desarrollo
    )